The purpose of the script is to classify georeferenced tiff files and isolate the yellow color. Organic soils are marked in yellow. Then, the classified areas of organic soils were polygonized into a shp file.

In [ ]:
import pandas as pd
import numpy as np
import os
from osgeo import osr, ogr
import cv2
!pip install gdal
from osgeo import gdal


In [ ]:
!pip install opencv-python-headless gdal numpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/MyDrive/Dane z Kaggla/GDAL/Raster/JPG/10JPG_without_georef ")
!ls

 clip_0001.dbf	 clip_0003.prj	 clip_0005.shp	 clip_0007.shx		  'Kopia wspolrzedne.txt'
 clip_0001.prj	 clip_0003.shp	 clip_0005.shx	 clip_0007.tif		   temp_georef_0001.tif
 clip_0001.shp	 clip_0003.shx	 clip_0005.tif	 coordinates_WGS84.txt	   temp_georef_0002.tif
 clip_0001.shx	 clip_0003.tif	 clip_0006.dbf	'Kopia mgsp2A0001.jpg'	   temp_georef_0003.tif
 clip_0001.tif	 clip_0004.dbf	 clip_0006.prj	'Kopia mgsp2A0002.jpg'	   temp_georef_0004.tif
 clip_0002.dbf	 clip_0004.prj	 clip_0006.shp	'Kopia mgsp2A0003.jpg'	   temp_georef_0005.tif
 clip_0002.prj	 clip_0004.shp	 clip_0006.shx	'Kopia mgsp2A0004.jpg'	   temp_georef_0006.tif
 clip_0002.shp	 clip_0004.shx	 clip_0006.tif	'Kopia mgsp2A0005.jpg'	   temp_georef_0007.tif
 clip_0002.shx	 clip_0004.tif	 clip_0007.dbf	'Kopia mgsp2A0006.jpg'
 clip_0002.tif	 clip_0005.dbf	 clip_0007.prj	'Kopia mgsp2A0007.jpg'
 clip_0003.dbf	 clip_0005.prj	 clip_0007.shp	'Kopia wspolrzedne1.txt'


Checking whether georeferencing has been assigned correctly. The geotransformation parameters of the file georeferenced manually in the QGIS program were compared with the parameters calculated automatically.

In [ ]:
GeotransformQGIS = gdal.Open('/content/drive/MyDrive/Dane z Kaggla/GDAL/Raster/JPG/3pliki z geo/mgsp2A0001_modified.tif')

In [ ]:
GeotransformQGIS.GetGeoTransform()

(368798.0393705895,
 6.336640020435696,
 0.0,
 781858.854831345,
 0.0,
 -6.336640020435696)

In [ ]:
GeotransformAutomat = gdal.Open('/content/drive/MyDrive/Dane z Kaggla/GDAL/Raster/JPG/7 JPG bez ref/georef_mgsp2A0001.tif')

In [ ]:
GeotransformAutomat.GetGeoTransform()

(368798.0393705895,
 6.336640020435696,
 0.0,
 781858.854831345,
 0.0,
 -6.336640020435696)

In [ ]:
GeotransformAutomat.GetProjection()

'PROJCS["ETRF2000-PL / CS92",GEOGCS["ETRF2000-PL",DATUM["ETRF2000_Poland",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","1305"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","9702"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",19],PARAMETER["scale_factor",0.9993],PARAMETER["false_easting",500000],PARAMETER["false_northing",-5300000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Northing",NORTH],AXIS["Easting",EAST],AUTHORITY["EPSG","2180"]]'

Input clipped tif files.

In [ ]:
src_Path = '/content/drive/MyDrive/Dane z Kaggla/GDAL/Raster/JPG/10JPG_without_georef '
input = [r for r in os.listdir(src_Path) if r.endswith('tif') and r.startswith('clip')]
input

['clip_0001.tif',
 'clip_0002.tif',
 'clip_0003.tif',
 'clip_0004.tif',
 'clip_0005.tif',
 'clip_0006.tif',
 'clip_0007.tif']

Output shp classified files.

In [ ]:
output_classified = []
for srcRst in input:
  dstRst = 'klas_' + srcRst[:-4] + '.shp'
  output_classified.append(dstRst)
output_classified

['klas_clip_0001.shp',
 'klas_clip_0002.shp',
 'klas_clip_0003.shp',
 'klas_clip_0004.shp',
 'klas_clip_0005.shp',
 'klas_clip_0006.shp',
 'klas_clip_0007.shp']

Classification

In [ ]:
import cv2
# import numpy as np
# from osgeo import ogr, osr, gdal

class YellowColorClassifier:
    def __init__(self, input_jpgs, output_shps, srs_epsg):
        self.input_jpgs = input_jpgs
        self.output_shps = output_shps
        self.srs_epsg = srs_epsg

    def classify_yellow(self, input_jpg):
        # Load the image
        image = cv2.imread(input_jpg)
        if image is None:
            raise ValueError(f"Could not open or find the image: {input_jpg}")

        # Convert the image to HSV color space
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Define the range for yellow color
        lower_yellow = np.array([20, 100, 100])
        upper_yellow = np.array([30, 255, 255])

        # Create a binary mask where yellow colors are white and other colors are black
        mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

        # Find contours in the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        return contours

    def get_geotransform(self, input_jpg):
        # Open the input JPEG file to get geotransformation
        jpeg_dataset = gdal.Open(input_jpg)
        if jpeg_dataset is None:
            raise ValueError(f"Could not open {input_jpg}")

        # Get geotransform
        geotransform = jpeg_dataset.GetGeoTransform()
        return geotransform

    def save_as_shapefile(self, contours, geotransform, output_shp):
        # Register the shapefile driver
        driver = ogr.GetDriverByName("ESRI Shapefile")
        if driver is None:
            raise ValueError("ESRI Shapefile driver not available")

        # Create the shapefile
        data_source = driver.CreateDataSource(output_shp)
        if data_source is None:
            raise ValueError(f"Could not create file {output_shp}")

        # Create the spatial reference, based on the provided EPSG code
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(self.srs_epsg)

        # Create the layer
        layer = data_source.CreateLayer("yellow_areas", srs, ogr.wkbPolygon)
        if layer is None:
            raise ValueError(f"Could not create layer")

        # Add an ID field
        field_id = ogr.FieldDefn("ID", ogr.OFTInteger)
        layer.CreateField(field_id)

        # Iterate through contours and create polygons
        for i, contour in enumerate(contours):
            # Create a polygon feature
            feature = ogr.Feature(layer.GetLayerDefn())
            feature.SetField("ID", i)

            # Create the polygon geometry from the contour points
            polygon = ogr.Geometry(ogr.wkbPolygon)
            ring = ogr.Geometry(ogr.wkbLinearRing)
            for point in contour:
                # Convert pixel coordinates to geographic coordinates using geotransform
                x = geotransform[0] + point[0][0] * geotransform[1] + point[0][1] * geotransform[2]
                y = geotransform[3] + point[0][0] * geotransform[4] + point[0][1] * geotransform[5]
                ring.AddPoint(float(x), float(y))
            ring.CloseRings()
            polygon.AddGeometry(ring)

            feature.SetGeometry(polygon)
            layer.CreateFeature(feature)

            # Destroy the feature to free resources
            feature.Destroy()

        # Destroy the data source to free resources
        data_source.Destroy()

    def run(self):
        for input_jpg, output_shp in zip(self.input_jpgs, self.output_shps):
            contours = self.classify_yellow(input_jpg)
            geotransform = self.get_geotransform(input_jpg)
            self.save_as_shapefile(contours, geotransform, output_shp)

In [ ]:
# Example usage
input_jpgs = input
output_shps = output_classified

srs_epsg = 4326  # EPSG code for the source georeferenced JPEG file

classifier = YellowColorClassifier(input_jpgs, output_shps, srs_epsg)
classifier.run()